# Mavlink Camera Walkthrough
>  Documentation 



In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev.showdoc import *
from fastcore.test import *


## Introduction
> Here we create an entire mavlink connection with client at the GCS and server at the camera. The client and server are connected via a UDP connection or a radio modem serial connection
> The camera can be controlled via the client, and the video stream is sent from the server to the client. The client can also request camera information, storage information, etc from the server.

In [ ]:
from UAV.mavlink.camera import CameraClient, CameraServer, MAVCom, boot_time_str
from UAV.mavlink.component import Component, mavutil, mavlink
from UAV.camera.fake_cam import CV2Camera, GSTCamera, read_camera_dict_from_toml   # fake camera for testing
from gstreamer import  GstPipeline
import gstreamer.utils as gst_utils
import time
from pathlib import Path
import cv2

Create a CameraClient and CameraServer

In [ ]:
MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA

Create a gstreamer pipelene to display the received video

Create Pysical camera object, here either CV2Camera or GSTCamera
The toml file contains the camera parameters, such as resolution, framerate, etc and also the gstreamer pipeline command to create the video streams.

In [ ]:
config_path = Path("../../config")
cam_gst_1 = GSTCamera(camera_dict=read_camera_dict_from_toml(config_path / "test_camera_info.toml"))
# cam_cv2_1 = CV2Camera(camera_dict=read_camera_dict_from_toml(config_path / "test_camera_info.toml"))

INFO   | uav.GSTCamera   | 21.972 | fake_cam.py:127 | MainThread         | GSTCamera Started
INFO   | pygst.GstVidSrc | 21.973 | gst_tools.py:131 | MainThread         | GstVidSrcValve 
 gst-launch-1.0 videotestsrc pattern=ball flip=true is-live=true num-buffers=1000 ! video/x-raw,framerate=10/1 !  tee name=t t. ! queue leaky=2 ! valve name=myvalve drop=False ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc tune=zerolatency ! rtph264pay ! udpsink host=127.0.0.1 port=5000 t. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=5/1,format=(string)BGR ! videoconvert ! appsink name=mysink emit-signals=true  sync=false async=false  max-buffers=2 drop=true 
INFO   | pygst.GstVidSrc | 21.982 | gst_tools.py:193 | MainThread         | Starting GstVidSrcValve
DEBUG  | pygst.GstVidSrc | 21.982 | gst_tools.py:197 | MainThread         | GstVidSrcValve Setting pipeline state to PLAYING ... 
DEBUG  | pygst.GstVidSrc | 21.983 | gst_tools.py:203 | Main

In [ ]:
SINK_PIPELINE = gst_utils.to_gst_string([
            'udpsrc port=5000 ! application/x-rtp, media=(string)video, clock-rate=(int)90000, encoding-name=(string)H264, payload=(int)96',
            'rtph264depay ! avdec_h264',
            'fpsdisplaysink',
            # 'autovideosink',
        ])

rcv_pipeline = GstPipeline(SINK_PIPELINE)     # Create a Gstreamer pipeline to display the received video on fpsdisplaysink

INFO   | pygst.GstPipeli | 21.990 | gst_tools.py:131 | MainThread         | GstPipeline 
 gst-launch-1.0 udpsrc port=5000 ! application/x-rtp, media=(string)video, clock-rate=(int)90000, encoding-name=(string)H264, payload=(int)96 ! rtph264depay ! avdec_h264 ! fpsdisplaysink


Create the client mavlink connection, this is mounted on the GCS

In [ ]:
# assert False

In [ ]:
client = MAVCom("udpin:localhost:14445", source_system=111, debug=False)  # for the client, we use the udpin connection, you can use serial as an option i.e "/dev/ttyACM0", "/dev/ttyUSB0"

INFO   | uav.MAVCom      | 22.131 |  mavcom.py:393 | Thread-5 (listen)  | MAVLink Mav2: True, source_system: 111


Create the server mavlink connection, this is mounted on the UAV

In [ ]:
server = MAVCom("udpout:localhost:14445", source_system=222, debug=False) # for the server, we use the udpout connection, you can use serial as an option  "/dev/ttyUSB0"

INFO   | uav.MAVCom      | 22.240 |  mavcom.py:393 | Thread-6 (listen)  | MAVLink Mav2: True, source_system: 222


Add the camera client to the client mavlink connection

In [ ]:
cam:CameraClient = client.add_component(
    CameraClient(mav_type=mavutil.mavlink.MAV_TYPE_CAMERA, source_component=11, debug=False))

INFO   | uav.CameraClien | 22.256 | component.py:111 | MainThread         | Component Started self.source_component = 11, self.mav_type = 30, self.source_system = 111


set_mav_connection CameraClient mavcom.py:107 self.mav_connection = <MAVCom>


Add the camera server to the server mavlink connection

In [ ]:
server.add_component(CameraServer(mav_type=mavutil.mavlink.MAV_TYPE_GCS, source_component=22, camera=cam_gst_1, debug=False))

INFO   | uav.CameraServe | 22.265 | component.py:111 | MainThread         | Component Started self.source_component = 22, self.mav_type = 6, self.source_system = 222


set_mav_connection CameraServer mavcom.py:107 self.mav_connection = <MAVCom>


<CameraServer>

Wait for the heartbeat from the camera server

In [ ]:
cam.wait_heartbeat(target_system=222, target_component=22, timeout=1)

True

Set the target system and component for the camera client
and request camera information, storage information, camera capture status, and camera settings

In [ ]:
cam.set_target(222, 22)  # client set target to server
cam.request_camera_information()
cam.request_storage_information()
cam.request_camera_capture_status()
cam.request_camera_settings()

CAMERA_Client  CAMERA_INFORMATION {time_boot_ms : 384, vendor_name : [74, 111, 104, 110, 32, 68, 111, 101, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], model_name : [70, 97, 107, 101, 32, 67, 97, 109, 101, 114, 97, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], firmware_version : 1, focal_length : 8.0, sensor_size_h : 6.0, sensor_size_v : 4.0, resolution_h : 1920, resolution_v : 1080, lens_id : 0, flags : 0, cam_definition_version : 1, cam_definition_uri : http://example.com/camera_definition.xml, gimbal_device_id : 0} 
CAMERA_Client  STORAGE_INFORMATION {time_boot_ms : 385, storage_id : 0, storage_count : 1, status : 0, total_capacity : 100000000.0, used_capacity : 0.0, available_capacity : 100000000.0, read_speed : 0.0, write_speed : 0.0, type : 0, name : } 
CAMERA_Client  CAMERA_CAPTURE_STATUS {time_boot_ms : 385, image_status : 0, video_status : 0, image_interval : 0.0, recording_time_ms : 0, 

<pymavlink.dialects.v20.ardupilotmega.MAVLink_camera_settings_message>

Start an image capture seqeunce, and display the images as they arrive

In [ ]:
cam.image_start_capture(interval=0.1, count=10)
while cam_gst_1.capture_thread.is_alive():
    if cam_gst_1.last_image is not None:
        cv2.imshow('gst_src', cam_gst_1.last_image)
        cam_gst_1.last_image = None
    cv2.waitKey(10)

Image saved to memory filesystem with name: 2023-09-17|06:59:22_0000.jpg
CAMERA_Client  CAMERA_IMAGE_CAPTURED {time_boot_ms : 399, time_utc : 1694890762300020, camera_id : 0, lat : 0, lon : 0, alt : 0, relative_alt : 0, q : [0.0, 0.0, 0.0, 0.0], image_index : 0, capture_result : 1, file_url : 2023-09-17|06:59:22_0000.jpg} 
Image saved to memory filesystem with name: 2023-09-17|06:59:22_0001.jpg
CAMERA_Client  CAMERA_IMAGE_CAPTURED {time_boot_ms : 503, time_utc : 1694890762404302, camera_id : 0, lat : 0, lon : 0, alt : 0, relative_alt : 0, q : [0.0, 0.0, 0.0, 0.0], image_index : 1, capture_result : 1, file_url : 2023-09-17|06:59:22_0001.jpg} 
Image saved to memory filesystem with name: 2023-09-17|06:59:22_0002.jpg
CAMERA_Client  CAMERA_IMAGE_CAPTURED {time_boot_ms : 606, time_utc : 1694890762507214, camera_id : 0, lat : 0, lon : 0, alt : 0, relative_alt : 0, q : [0.0, 0.0, 0.0, 0.0], image_index : 2, capture_result : 1, file_url : 2023-09-17|06:59:22_0002.jpg} 
Image saved to memory fil

Shutdown the receive pipeline and close the mavlink connections

In [ ]:
rcv_pipeline.shutdown()
client.close()
server.close()
cv2.destroyAllWindows()

INFO   | pygst.GstPipeli | 24.046 | gst_tools.py:264 | MainThread         | GstPipeline Shutdown requested ...
INFO   | pygst.GstPipeli | 24.046 | gst_tools.py:268 | MainThread         | GstPipeline successfully destroyed
INFO   | uav.CameraClien | 25.259 | component.py:366 | MainThread         | CameraClient closed
INFO   | uav.MAVCom      | 25.261 |  mavcom.py:441 | MainThread         | MAVCom  closed
INFO   | pygst.GstVidSrc | 26.262 | gst_tools.py:264 | MainThread         | GstVidSrcValve Shutdown requested ...
DEBUG  | pygst.GstVidSrc | 26.263 | gst_tools.py:234 | MainThread         | GstVidSrcValve Stopping pipeline ...
DEBUG  | pygst.GstVidSrc | 26.265 | gst_tools.py:238 | MainThread         | GstVidSrcValve Sending EOS event ...
DEBUG  | pygst.GstVidSrc | 26.268 | gst_tools.py:248 | MainThread         | GstVidSrcValve Reseting pipeline state ....
DEBUG  | pygst.GstVidSrc | 26.271 | gst_tools.py:255 | MainThread         | GstVidSrcValve Gst.Pipeline successfully destroyed
INFO  

In [ ]:
#| hide
# assert False

Perform the same test, but with the CV2Camera all in one cell

In [ ]:
from UAV.mavlink.camera import CameraClient, CameraServer, MAVCom, boot_time_str
from UAV.mavlink.component import Component, mavutil, mavlink
from UAV.camera.fake_cam import CV2Camera, GSTCamera, read_camera_dict_from_toml   # fake camera for testing
from gstreamer import  GstPipeline
import gstreamer.utils as gst_utils
import time
from pathlib import Path
import cv2

SINK_PIPELINE = gst_utils.to_gst_string([
            'udpsrc port=5000 ! application/x-rtp, media=(string)video, clock-rate=(int)90000, encoding-name=(string)H264, payload=(int)96',
            'rtph264depay ! avdec_h264',
            'fpsdisplaysink',
            # 'autovideosink',
        ])
con1, con2 = "udpin:localhost:14445", "udpout:localhost:14445"
# con1, con2 = "/dev/ttyACM0", "/dev/ttyUSB0"

print (f"{boot_time_str =}")
config_path = Path("../../config")

cam_uav = GSTCamera(camera_dict=read_camera_dict_from_toml(config_path / "test_camera_info.toml"))
# cam_uav = CV2Camera(camera_dict=read_camera_dict_from_toml(config_path / "test_camera_info.toml"))
with GstPipeline(SINK_PIPELINE) as rcv_pipeline:     # Create a Gstreamer pipeline to display the received video on fpsdisplaysink
    with MAVCom(con1, source_system=111, debug=False) as client:
        with MAVCom(con2, source_system=222, debug=False) as server:
            cam_gcs:CameraClient = client.add_component(
                CameraClient(mav_type=mavutil.mavlink.MAV_TYPE_GCS, source_component=11, debug=False))
            server.add_component(CameraServer(mav_type=mavutil.mavlink.MAV_TYPE_CAMERA, source_component=22, camera=cam_uav, debug=False))
            # server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=None, debug=False))

            cam_gcs.wait_heartbeat(target_system=222, target_component=22, timeout=1)
            time.sleep(0.1)
            
            cam_gcs.set_target(222, 22) # client set target to server
            cam_gcs.request_camera_information()
            cam_gcs.request_storage_information()
            cam_gcs.request_camera_capture_status()
            cam_gcs.request_camera_information()
            cam_gcs.request_camera_settings()
            cam_gcs.image_start_capture(interval=0.1, count=10)

            while cam_uav.capture_thread.is_alive():
                if cam_uav.last_image is not None:
                    cv2.imshow('gst_src', cam_uav.last_image)
                    cam_uav.last_image = None
                cv2.waitKey(10)

            start = time.time()
            msg = cam_gcs.request_storage_information()
            print (msg)
            print(f"request_storage_information time : {1000*(time.time() - start) = } msec")
            
cv2.destroyAllWindows()


INFO   | uav.GSTCamera   | 30.892 | fake_cam.py:127 | MainThread         | GSTCamera Started
INFO   | pygst.GstVidSrc | 30.893 | gst_tools.py:131 | MainThread         | GstVidSrcValve 
 gst-launch-1.0 videotestsrc pattern=ball flip=true is-live=true num-buffers=1000 ! video/x-raw,framerate=10/1 !  tee name=t t. ! queue leaky=2 ! valve name=myvalve drop=False ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc tune=zerolatency ! rtph264pay ! udpsink host=127.0.0.1 port=5000 t. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=5/1,format=(string)BGR ! videoconvert ! appsink name=mysink emit-signals=true  sync=false async=false  max-buffers=2 drop=true 
INFO   | pygst.GstVidSrc | 30.900 | gst_tools.py:193 | MainThread         | Starting GstVidSrcValve
DEBUG  | pygst.GstVidSrc | 30.900 | gst_tools.py:197 | MainThread         | GstVidSrcValve Setting pipeline state to PLAYING ... 
DEBUG  | pygst.GstVidSrc | 30.901 | gst_tools.py:203 | Main

boot_time_str ='2023-09-17|07:01:30'


INFO   | uav.MAVCom      | 31.174 |  mavcom.py:393 | Thread-6 (listen)  | MAVLink Mav2: True, source_system: 222
INFO   | uav.CameraClien | 31.215 | component.py:111 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
INFO   | uav.CameraServe | 31.216 | component.py:111 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222


set_mav_connection CameraClient mavcom.py:107 self.mav_connection = <MAVCom>
set_mav_connection CameraServer mavcom.py:107 self.mav_connection = <MAVCom>
CAMERA_Client  CAMERA_INFORMATION {time_boot_ms : 476, vendor_name : [74, 111, 104, 110, 32, 68, 111, 101, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], model_name : [70, 97, 107, 101, 32, 67, 97, 109, 101, 114, 97, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], firmware_version : 1, focal_length : 8.0, sensor_size_h : 6.0, sensor_size_v : 4.0, resolution_h : 1920, resolution_v : 1080, lens_id : 0, flags : 0, cam_definition_version : 1, cam_definition_uri : http://example.com/camera_definition.xml, gimbal_device_id : 0} 
CAMERA_Client  STORAGE_INFORMATION {time_boot_ms : 477, storage_id : 0, storage_count : 1, status : 0, total_capacity : 100000000.0, used_capacity : 0.0, available_capacity : 100000000.0, read_speed : 0.0, write_speed : 0.0, type 

WARNIN | pygst.GstPipeli | 31.425 | gst_tools.py:289 | MainThread         | Gstreamer.GstPipeline: gst-core-error-quark: Pipeline construction is invalid, please add queues. (13). ../libs/gst/base/gstbasesink.c(1249): gst_base_sink_query_latency (): /GstPipeline:pipeline1/GstFPSDisplaySink:fpsdisplaysink0/GstAutoVideoSink:fps-display-video_sink/GstXvImageSink:fps-display-video_sink-actual-sink-xvimage:
Not enough buffering available for  the processing deadline of 0:00:00.015000000, add enough queues to buffer  0:00:00.015000000 additional data. Shortening processing latency to 0:00:00.000000000.


Image saved to memory filesystem with name: 2023-09-17|07:01:31_0001.jpg
CAMERA_Client  CAMERA_IMAGE_CAPTURED {time_boot_ms : 585, time_utc : 1694890891427054, camera_id : 0, lat : 0, lon : 0, alt : 0, relative_alt : 0, q : [0.0, 0.0, 0.0, 0.0], image_index : 1, capture_result : 1, file_url : 2023-09-17|07:01:31_0001.jpg} 
Image saved to memory filesystem with name: 2023-09-17|07:01:31_0002.jpg
CAMERA_Client  CAMERA_IMAGE_CAPTURED {time_boot_ms : 689, time_utc : 1694890891530324, camera_id : 0, lat : 0, lon : 0, alt : 0, relative_alt : 0, q : [0.0, 0.0, 0.0, 0.0], image_index : 2, capture_result : 1, file_url : 2023-09-17|07:01:31_0002.jpg} 
Image saved to memory filesystem with name: 2023-09-17|07:01:31_0003.jpg
CAMERA_Client  CAMERA_IMAGE_CAPTURED {time_boot_ms : 803, time_utc : 1694890891644273, camera_id : 0, lat : 0, lon : 0, alt : 0, relative_alt : 0, q : [0.0, 0.0, 0.0, 0.0], image_index : 3, capture_result : 1, file_url : 2023-09-17|07:01:31_0003.jpg} 
Image saved to memory fil

INFO   | pygst.GstVidSrc | 33.219 | gst_tools.py:264 | MainThread         | GstVidSrcValve Shutdown requested ...
DEBUG  | pygst.GstVidSrc | 33.220 | gst_tools.py:234 | MainThread         | GstVidSrcValve Stopping pipeline ...
DEBUG  | pygst.GstVidSrc | 33.221 | gst_tools.py:238 | MainThread         | GstVidSrcValve Sending EOS event ...
DEBUG  | pygst.GstVidSrc | 33.224 | gst_tools.py:248 | MainThread         | GstVidSrcValve Reseting pipeline state ....
DEBUG  | pygst.GstVidSrc | 33.228 | gst_tools.py:255 | MainThread         | GstVidSrcValve Gst.Pipeline successfully destroyed
INFO   | pygst.GstVidSrc | 33.229 | gst_tools.py:268 | MainThread         | GstVidSrcValve successfully destroyed
INFO   | uav.GSTCamera   | 33.229 | fake_cam.py:262 | MainThread         | GSTCamera closed
INFO   | uav.CameraServe | 34.219 | component.py:366 | MainThread         | CameraServer closed
INFO   | uav.MAVCom      | 34.221 |  mavcom.py:441 | MainThread         | MAVCom  closed
INFO   | uav.CameraCli